# 04 - Treinamento dos Modelos com Ensemble

Neste notebook, é realizado o treinamento e avaliação de modelos ensemble.

## Importação

In [1]:
# Bibliotecas padrão
import pickle
import warnings
import os
from datetime import datetime as dt
from collections import Counter

# Bibliotecas utilitárias de terceiros
import numpy as np
import pandas as pd
import scipy.stats as ss
from tqdm.notebook import tqdm

# Classes do modelo de aprendizado
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
# Funções de avaliação e seleção de modelos
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
# Para escalonamento de dados e pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
# Para utilização de bagging
from sklearn.ensemble import BaggingClassifier
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.base import clone

# Gráficos
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
from plotly.subplots import make_subplots

## Constantes e Sets

In [2]:
# PALETTE = 'RdYlGn'
# PALETTE = 'viridis'
# PALETTE = 'inferno'
PALETTE = 'balance'
BLUE = '#346CB5'
SEED = 42
N_ITER = 20  # Podemos diminuir
TRAINVAL_SPLITS = 3  # Parece um número bom pra mim! 3, 5, 10...
FONT_SIZE = 16
EPS = np.finfo(float).eps

warnings.simplefilter('ignore'); os.environ['PYTHONWARNINGS'] = 'ignore'  # Para remover warnings desnecessários que só poluem o notebook
pd.set_option('display.max_columns', None)
np.random.seed(SEED)

pio.templates['bigger-font'] = pio.templates['plotly']
pio.templates['bigger-font']['layout']['font']['size'] = FONT_SIZE  # Usado para exportar para png

In [3]:
KNN_BEST_PARAMS = {
    'model__n_neighbors': 5,
    'model__p': 1,
    'model__weights': 'distance'    
}

DT_BEST_PARAMS = {
    'max_depth': None,
    'min_samples_leaf': 1,
    'min_samples_split': 2    
}

LR_BEST_PARAMS = {
    'model__solver': 'saga',
    'model__penalty': 'l1',
    'model__C': 1
}

SVC_PARAM_GRID = {
    'model__C': 10,
    'model__kernel': 'rbf',
    'model__gamma': 'scale'
}

MLP_PARAM_GRID = {
    'model__hidden_layer_sizes': (50,),
    'model__activation': 'tanh',
    'model__solver': 'adam',
    'model__alpha': 0.0001,
    'model__learning_rate': 'constant'
}


BAGGING_PARAM_GRID = {
    'n_estimators': [10, 50, 100, 200]
}

## Funções

In [4]:
def generate_colors(num_colors):
    colors = px.colors.sample_colorscale(PALETTE, [n/(num_colors - 1) for n in range(num_colors)])
    
    return colors

In [5]:
def validate_ensemble(model, X, y, param_grid, targets_map={}, n_iter=10, n_trainval_splits=10):
    skf = StratifiedKFold(n_splits=n_iter, random_state=SEED, shuffle=True)   # Shuffle?
    skf_folds = skf.split(X, y)
    
    runs_metrics = {}

    for n, (trainval_idx, test_idx) in enumerate(tqdm(skf_folds, total=n_iter)):
        X_trainval = X[trainval_idx]
        y_trainval = y[trainval_idx]
        
        X_test = X[test_idx]
        y_test = y[test_idx]
    
        best_params = get_best_params(model, X_trainval, y_trainval, param_grid, n_trainval_splits=n_trainval_splits, 
                                      display_results=False)

        t0_t = dt.now()
        model.set_params(**best_params)
        model.fit(X_trainval, y_trainval)    
        tf_t = dt.now()
        tt_delta = tf_t - t0_t
        
        model_metrics = evaluate_model_performance(model, X_test, y_test, targets_map=targets_map)
        model_metrics['best_params'] = best_params
        model_metrics['training_time'] = tt_delta
        runs_metrics[n] = model_metrics
    
    runs_metrics = aggregate_run_metrics(runs_metrics)
    return runs_metrics

In [83]:
def compute_feasibility(base_model, X, y, n_runs=3, n_splits=20, n_estimators=100):
    skf = StratifiedKFold(n_splits=n_splits, random_state=SEED, shuffle=True)  # Shuffle?
    skf_folds = skf.split(X, y)
    train_idx, test_idx = next(skf_folds) #apenas pava avaliar uma das runs do problema original

    bagging_model = BaggingClassifier(base_model, n_estimators=n_estimators)

    fit_times = []
    pred_times = []
    for _ in range(n_runs):
        X_train = X[train_idx]
        y_train = y[train_idx]
        
        X_test = X[test_idx]
        #y_test = y[test_idx]
                
        t0_f = dt.now()
        bagging_model.fit(X_train, y_train)
        tf_f = dt.now()
        tf_delta = (tf_f - t0_f).total_seconds()
        fit_times.append(tf_delta)

        t0_p = dt.now()
        _ = bagging_model.predict(X_test) # não associado para nenhuma variável pois só queremos avaliar o tempo, não o resultado
        tf_p = dt.now()
        tp_delta = (tf_p - t0_p).total_seconds()
        pred_times.append(tp_delta)

    feasibility_results = {'mean_pred_time': np.mean(pred_times),
                           'std_pred_time': np.std(pred_times),
                           'mean_fit_time': np.mean(fit_times),
                           'std_fit_time': np.std(fit_times)
                           }

    return feasibility_results

In [7]:
def get_best_params(model, X_trainval, y_trainval, param_grid, n_trainval_splits=10, display_results=False):
    skf = StratifiedKFold(n_splits=n_trainval_splits, random_state=SEED, shuffle=True)  # Shuffle?
    
    grid_search = GridSearchCV(model, param_grid=param_grid, refit=False, cv=skf, n_jobs=-1)
    grid_search.fit(X_trainval, y_trainval)
    
    best_params = grid_search.best_params_

    if display_results:
        df_res = pd.DataFrame(grid_search.cv_results_)
        df_res = df_res.sort_values('rank_test_score', ascending=True)
        display(df_res)

    return best_params

In [8]:
def evaluate_model_performance(model, X, y, targets_map={}):
    t0_p = dt.now()
    y_pred = model.predict(X)
    tf_p = dt.now()
    tp_delta = tf_p - t0_p
    
    cm = confusion_matrix(y, y_pred)

    if targets_map != {}:
        target_names = [targets_map[yy] for yy in np.unique(y)]

    else:
        target_names = None
            
    report_dict = classification_report(y, y_pred, output_dict=True, target_names=target_names)
    report_dict['cm'] = cm
    report_dict['prediction_time'] = tp_delta

    return report_dict

In [9]:
def aggregate_run_metrics(runs_res):
    runs_ids = runs_res.keys()
    labels = [label for label in runs_res[0].keys() if label not in ['accuracy', 'micro avg', 'best_params', 'macro avg', 
                                                                     'weighted avg', 'training_time', 'prediction_time', 'cm']]  # Porco?

    accuracies = [runs_res[i]['accuracy'] for i in runs_ids]
    cms = [runs_res[i]['cm'] for i in runs_ids]
    f1_scores = {i: [runs_res[j][i]['f1-score'] for j in runs_ids] for i in labels}
    recalls = {i: [runs_res[j][i]['recall'] for j in runs_ids] for i in labels}
    precisions = {i: [runs_res[j][i]['precision'] for j in runs_ids] for i in labels}
    best_params = [runs_res[i]['best_params'] for i in runs_ids]
    training_time = [runs_res[i]['training_time'] for i in runs_ids]
    prediction_time = [runs_res[i]['prediction_time'] for i in runs_ids]

    metrics = {
        'accuracies': accuracies,
        'cms': cms,
        'f1-scores': f1_scores,
        'recalls': recalls,
        'precisions': precisions,
        'best_params': best_params,
        'training_time': training_time,
        'prediction_time': prediction_time
      }

    return metrics

In [10]:
def print_res(res):  # Cumulo da porquice, mas funciona bem
    mean_acc = np.mean(res['accuracies']) * 100
    std_acc = np.std(res['accuracies'], ddof=1) * 100
    
    print(f"===> ACURÁCIA MÉDIA <===\n({mean_acc:.2f} ± {std_acc:.2f})%", end='\n\n')
    print(f"===> MATRIZ DE CONFUSÃO GERAL <===\n{np.sum(res['cms'], axis=0)}", end='\n\n')
    
    print('===> RECALL, PRECISION E F1-SCORE MÉDIO <===') 
    print(f"{'Target'.ljust(12)} | {'Recall'.ljust(12)} (%) | {'Precision'.ljust(12)} (%) | {'F1-Score'.ljust(12)} (%)")
    print('-'*70)
    mean_recalls = []
    mean_precisions = []
    mean_f1_scores = []
    for l in res['recalls'].keys():  # Igual para todos
        mean_recall = np.mean(res['recalls'][l]) * 100
        mean_precision = np.mean(res['precisions'][l]) * 100
        mean_f1_score = np.mean(res['f1-scores'][l]) * 100

        std_recall = np.std(res['recalls'][l], ddof=1) * 100
        std_precision = np.std(res['precisions'][l], ddof=1) * 100
        std_f1_score = np.std(res['f1-scores'][l], ddof=1) * 100

        mean_recalls.append(mean_recall)
        mean_precisions.append(mean_precision)
        mean_f1_scores.append(mean_f1_score)
        
        print(f'{l.ljust(12)} | '
              f'{(str(np.round(mean_recall, 2)) + " ± " + str(np.round(std_recall, 2))).ljust(16)} | '
              f'{(str(np.round(mean_precision, 2)) + " ± " + str(np.round(std_precision, 2))).ljust(16)} | '
              f'{(str(np.round(mean_f1_score, 2)) + " ± " + str(np.round(std_f1_score, 2))).ljust(16)}')

    std_recalls = np.std(mean_recalls, ddof=1)
    std_precisions = np.std(mean_precisions, ddof=1)
    std_f1_scores = np.std(mean_f1_scores, ddof=1)
    
    mean_recalls = np.mean(mean_recalls)
    mean_precisions = np.mean(mean_precisions)
    mean_f1_scores = np.mean(mean_f1_scores)
    
    print(f"\n===> MÉDIA DO RECALL MÉDIO <===\n({mean_recalls:.2f} ± {std_recalls:.2f})%", end='\n\n')
    print(f"===> MÉDIA DO PRECISION MÉDIO <===\n({mean_precisions:.2f} ± {std_precisions:.2f})%", end='\n\n')
    print(f"===> MÉDIA DO F1-SCORE MÉDIO <===\n({mean_f1_scores:.2f} ± {std_f1_scores:.2f})%", end='\n\n')

    print('\n===> MELHORES HIPERPARÂMETROS <===') 
    print(f"{'Ocorrências'.ljust(12)} | {'Valores'.ljust(75)}")
    print('-'*130)
    params_counts = Counter(tuple(param.items()) for param in res['best_params'])
    params_counts_mc = params_counts.most_common()
    
    for pcm in params_counts_mc:  # [:5] para mostrar apenas top 5
        pcm_values = pcm[0]
        pcm_occ = pcm[1]
    
        print(f'{str(pcm_occ).ljust(12)} | {pcm_values}')

    print(f"\n===> TEMPO DE TREINAMENTO MÉDIO <===\n{np.mean(res['training_time'])}")

    print(f"\n===> TEMPO DE INFERÊNCIA MÉDIO <===\n{np.mean(res['prediction_time'])}")

In [11]:
def plot_accuracies(res, export=False, filename='acc', path='.'):
    accuracies = [r*100 for r in res['accuracies']]  # Acho que fica melhor em percentual
    
    fig = go.Figure()
    fig.add_trace(go.Histogram(x=accuracies, nbinsx=5, marker_color=BLUE))
    fig.add_vline(x=np.mean(accuracies), line_dash='dash', annotation_text=f' Acurácia Média: {np.mean(accuracies):.2f}%')
    fig.update_layout(title=f'Distribuição das Acurácias após {N_ITER} iterações', yaxis_title='Frequência', xaxis_title='Acurácia (%)',
                      height=600, autosize=True)

    if export:
        export_fig(fig, filename, path)
    
    fig.show()

In [12]:
def plot_label_metrics(res, metric, export=False, filename='label_metric', path='.'):
    label_score = {k: [v*100 for v in res[metric][k]] for k in res[metric].keys()}  # Percentual
    
    labels = list(label_score.keys())
    marker_colors = generate_colors(len(labels))

    fig = go.Figure()
    for i, l in enumerate(labels):
        color = marker_colors[i]
        f1_score = label_score[l]

        fig.add_trace(go.Box(y=f1_score, name=l, marker_color=color, legendgroup=i))#, boxpoints='all'))

    fig.update_layout(title=f'Boxplots de {metric.title()} por label após {N_ITER} iterações', yaxis_title=f"{metric.title()} (%)",
                      xaxis_title="Classes", yaxis_range=(0, 101), height=600, autosize=True)  # Para ter comparabilidade entre os modelos

    if export:
        export_fig(fig, filename, path)
    
    fig.show()

In [13]:
def plot_confusion_matrix(res, export=False, filename='cm', path='.'):
    cms = res['cms']
    axis_labels = list(res['recalls'].keys())  # Igual para todos
    
    cm = np.sum(cms, axis=0)
    cm_mean = np.mean(cms, axis=0)
    cm_recall = cm/cm.sum(axis=1, keepdims=True)  # Normalização pela linha
    cm_precision = cm/cm.sum(axis=0, keepdims=True)  # Normalização pela coluna

    fig = make_subplots(rows=2, cols=2, shared_xaxes=True, shared_yaxes=True, vertical_spacing=0.1,
                        subplot_titles=['Matriz de Confusão agregada', 'Matriz de Confusão média', 
                                        'Matriz de Confusão normalizada pelas linhas', 
                                        'Matriz de Confusão normalizada pelas colunas'])
    
    fig.add_trace(go.Heatmap(x=axis_labels, y=axis_labels, z=cm, text=cm, texttemplate='%{text}', 
                             showscale=False, colorscale=PALETTE), 
                  row=1, col=1)
    fig.add_trace(go.Heatmap(x=axis_labels, y=axis_labels, z=cm_mean, text=cm_mean, texttemplate='%{text:.2f}', 
                             showscale=False, colorscale=PALETTE), 
                  row=1, col=2)


    fig.add_trace(go.Heatmap(x=axis_labels, y=axis_labels, z=cm_recall, text=cm_recall, texttemplate='%{text:.2f}', 
                             showscale=False, colorscale=PALETTE), 
                  row=2, col=1)
    fig.add_trace(go.Heatmap(x=axis_labels, y=axis_labels, z=cm_precision, text=cm_precision, texttemplate='%{text:.2f}', 
                             showscale=False, colorscale=PALETTE), 
                  row=2, col=2)

    fig.update_layout(title=f'Matrizes de Confusão geradas após {N_ITER} iterações', yaxis1_title='Real', yaxis3_title='Real', 
                      xaxis3_title='Predito', xaxis4_title='Predito', yaxis1_autorange='reversed', yaxis2_autorange='reversed', 
                      yaxis3_autorange='reversed', height=1000, autosize=True)

    if export:
        export_fig(fig, filename, path)

    fig.show()

In [14]:
def export_fig(fig, filename, path):
    fig_png = go.Figure(fig)  # Deep copy para não alterar o objeto original
    fig_html = go.Figure(fig)

    fig_png = fig_png.update_layout(template='bigger-font', width=1100)  # Para cobrir toda lateral da pag
    fig_html = fig_html.update_layout(width=None, height=None, autosize=True)  # Para mudar conforme a página html

    fig_png.write_image(f'{path}/{filename}.png', scale=3)
    fig_html.write_html(f'{path}/{filename}.html')

## Scripts

### Leitura

#### Base de dados

Como primeiro passo vamos realizar a leitura dos dados pré-tratados anteriormente.

In [15]:
df = pd.read_pickle('../data/processed/steel-plates-fault.pkl')
df

,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,Length_of_Conveyer,TypeOfSteel_A300,TypeOfSteel_A400,Steel_Plate_Thickness,Edges_Index,Empty_Index,Square_Index,Outside_X_Index,Edges_X_Index,Edges_Y_Index,Outside_Global_Index,LogOfAreas,Log_X_Index,Log_Y_Index,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Target
0,42.0,50.0,270900.0,270944.0,267.0,17.0,44.0,24220.0,76.0,108.0,1687.0,1.0,0.0,80.0,0.0498,0.2415,0.1818,0.0047,0.4706,1.0000,1.0,2.4265,0.9031,1.6435,0.8182,-0.2913,0.5822,0
1,645.0,651.0,2538079.0,2538108.0,108.0,10.0,30.0,11397.0,84.0,123.0,1687.0,1.0,0.0,80.0,0.7647,0.3793,0.2069,0.0036,0.6000,0.9667,1.0,2.0334,0.7782,1.4624,0.7931,-0.1756,0.2984,0
2,829.0,835.0,1553913.0,1553931.0,71.0,8.0,19.0,7972.0,99.0,125.0,1623.0,1.0,0.0,100.0,0.9710,0.3426,0.3333,0.0037,0.7500,0.9474,1.0,1.8513,0.7782,1.2553,0.6667,-0.1228,0.2150,0
3,853.0,860.0,369370.0,369415.0,176.0,13.0,45.0,18996.0,99.0,126.0,1353.0,0.0,1.0,290.0,0.7287,0.4413,0.1556,0.0052,0.5385,1.0000,1.0,2.2455,0.8451,1.6532,0.8444,-0.1568,0.5212,0
4,1289.0,1306.0,498078.0,498335.0,2409.0,60.0,260.0,246930.0,37.0,126.0,1353.0,0.0,1.0,185.0,0.0695,0.4486,0.0662,0.0126,0.2833,0.9885,1.0,3.3818,1.2305,2.4099,0.9338,-0.1992,1.0000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1263,221.0,242.0,3948212.0,3948253.0,519.0,33.0,41.0,48309.0,65.0,124.0,1360.0,1.0,0.0,200.0,0.3250,0.3972,0.5122,0.0154,0.6364,1.0000,1.0,2.7152,1.3222,1.6128,0.4878,-0.2728,0.9765,5
1264,1111.0,1121.0,4032298.0,4032320.0,110.0,20.0,22.0,12351.0,100.0,127.0,1354.0,1.0,0.0,200.0,0.3442,0.5000,0.4545,0.0074,0.5000,1.0000,1.0,2.0414,1.0000,1.3424,0.5454,-0.1228,0.3663,5
1265,995.0,1006.0,4085316.0,4085344.0,140.0,25.0,28.0,16076.0,103.0,132.0,1356.0,1.0,0.0,200.0,0.5162,0.5454,0.3929,0.0081,0.4400,1.0000,1.0,2.1461,1.0414,1.4472,0.6071,-0.1029,0.5096,5
1266,396.0,418.0,4116853.0,4116868.0,231.0,26.0,16.0,25096.0,56.0,141.0,1356.0,1.0,0.0,200.0,0.5841,0.3000,0.6818,0.0162,0.8461,0.9375,0.0,2.3636,1.3424,1.1761,-0.3182,-0.1512,0.5461,5


In [16]:
df.dtypes

X_Minimum                float64
X_Maximum                float64
Y_Minimum                float64
Y_Maximum                float64
Pixels_Areas             float64
X_Perimeter              float64
Y_Perimeter              float64
Sum_of_Luminosity        float64
Minimum_of_Luminosity    float64
Maximum_of_Luminosity    float64
Length_of_Conveyer       float64
TypeOfSteel_A300         float64
TypeOfSteel_A400         float64
Steel_Plate_Thickness    float64
Edges_Index              float64
Empty_Index              float64
Square_Index             float64
Outside_X_Index          float64
Edges_X_Index            float64
Edges_Y_Index            float64
Outside_Global_Index     float64
LogOfAreas               float64
Log_X_Index              float64
Log_Y_Index              float64
Orientation_Index        float64
Luminosity_Index         float64
SigmoidOfAreas           float64
Target                     int64
dtype: object

In [17]:
df.describe()

,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,Length_of_Conveyer,TypeOfSteel_A300,TypeOfSteel_A400,Steel_Plate_Thickness,Edges_Index,Empty_Index,Square_Index,Outside_X_Index,Edges_X_Index,Edges_Y_Index,Outside_Global_Index,LogOfAreas,Log_X_Index,Log_Y_Index,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Target
count,1268.000000,1268.000000,1.268000e+03,1.268000e+03,1268.000000,1268.000000,1268.000000,1.268000e+03,1268.000000,1268.000000,1268.000000,1268.000000,1268.000000,1268.000000,1268.000000,1268.000000,1268.000000,1268.000000,1268.000000,1268.000000,1268.000000,1268.000000,1268.000000,1268.000000,1268.000000,1268.000000,1268.00000,1268.000000
mean,508.635647,565.126183,1.759791e+06,1.759855e+06,2588.908517,143.205836,103.716088,2.831348e+05,79.209779,130.299685,1445.250789,0.402997,0.597003,66.072555,0.310388,0.409378,0.589186,0.040520,0.597253,0.802600,0.562303,2.603358,1.388906,1.456804,0.067459,-0.135566,0.61065,2.695584
std,513.760815,481.284469,1.719432e+06,1.719449e+06,6107.392220,362.087198,524.113646,6.006898e+05,32.625475,16.654437,137.299471,0.490694,0.490694,35.923484,0.303147,0.134019,0.264234,0.062152,0.241613,0.241219,0.484225,0.880245,0.535012,0.468135,0.483909,0.137824,0.34960,1.802939
min,0.000000,6.000000,7.430000e+03,7.458000e+03,2.000000,2.000000,1.000000,2.500000e+02,0.000000,70.000000,1227.000000,0.000000,0.000000,40.000000,0.000000,0.000000,0.008300,0.001500,0.014400,0.105000,0.000000,0.301000,0.301000,0.000000,-0.931900,-0.998900,0.11900,0.000000
25%,41.000000,191.000000,6.266300e+05,6.266358e+05,87.000000,15.000000,13.000000,1.011000e+04,46.000000,124.000000,1358.000000,0.000000,0.000000,40.000000,0.058500,0.314850,0.375700,0.006600,0.400000,0.539850,0.000000,1.939500,1.000000,1.079200,-0.364800,-0.192625,0.24820,1.000000
50%,283.000000,330.500000,1.412536e+06,1.412546e+06,200.500000,28.000000,28.000000,2.135150e+04,85.500000,127.000000,1362.000000,0.000000,1.000000,60.000000,0.181450,0.407450,0.571400,0.010150,0.633500,0.956500,1.000000,2.302100,1.176100,1.389050,0.072250,-0.143000,0.57080,2.000000
75%,955.000000,963.250000,2.246608e+06,2.246677e+06,3638.500000,184.250000,115.000000,3.696388e+05,103.000000,140.000000,1624.000000,1.000000,1.000000,70.000000,0.533050,0.493725,0.837025,0.067625,0.777800,1.000000,1.000000,3.560900,2.011750,1.812900,0.468275,-0.081650,1.00000,5.000000
max,1688.000000,1696.000000,1.298766e+07,1.298769e+07,152655.000000,10449.000000,18152.000000,1.159141e+07,196.000000,252.000000,1794.000000,1.000000,1.000000,290.000000,0.992300,0.943900,1.000000,0.622600,1.000000,1.000000,1.000000,5.183700,2.938500,4.258700,0.991700,0.591700,1.00000,5.000000


In [18]:
columns = df.drop(columns=['Target']).columns
colors = generate_colors(len(columns))

fig = go.Figure()
for column, color in zip(columns, colors):
    fig.add_trace(go.Histogram(x=df[column], name=column, marker_color=color, opacity=0.5))
    
fig.update_layout(title='Distribuição das variáveis', xaxis_title='Valor', yaxis_title='Frequência', barmode='overlay', height=600)
fig.show()

#### Mapa dos targets

Em seguida, iremos realizar a importação do mapa dos targets. Isto irá facilitar na leitura dos gráficos e validação do problema:

In [19]:
with open('../data/processed/target_maps.pkl', 'rb') as file: 
    targets_map = pickle.load(file)

targets_map

{0: 'Pastry',
 1: 'Z_Scratch',
 2: 'K_Scatch',
 3: 'Stains',
 4: 'Dirtiness',
 5: 'Bumps',
 6: 'Other_Faults'}

### Divisão

Para este projeto, utilizaremos todas as variáveis disponíveis (exceto `target`) para realizar as predições. Assim, teremos:

In [20]:
X = df.drop(columns=['Target']).values
y = df['Target'].values

X.shape, y.shape

((1268, 27), (1268,))

### Instanciação dos modelos base

In [21]:
knn_base = Pipeline([('scaler', StandardScaler()) , ('model', KNeighborsClassifier())])
knn_base.set_params(**KNN_BEST_PARAMS)

dt_base = DecisionTreeClassifier(random_state=SEED)
dt_base.set_params(**DT_BEST_PARAMS)

lr_base = Pipeline([('scaler', StandardScaler()), ('model', LogisticRegression(random_state=SEED, max_iter=5000))])
lr_base.set_params(**LR_BEST_PARAMS)

svc_base = Pipeline([('scaler', StandardScaler()), ('model', SVC(random_state=SEED))])
svc_base.set_params(**SVC_PARAM_GRID)

mlp_base = Pipeline([('scaler', StandardScaler()), ('model', MLPClassifier(random_state=SEED, max_iter=100_000, validation_fraction=0.2,
                                                                            n_iter_no_change=200, tol=0.0001, early_stopping=True))])
mlp_base.set_params(**MLP_PARAM_GRID)

Pipeline(steps=[('scaler', StandardScaler()),
                ('model',
                 MLPClassifier(activation='tanh', early_stopping=True,
                               hidden_layer_sizes=(50,), max_iter=100000,
                               n_iter_no_change=200, random_state=42,
                               validation_fraction=0.2))])

### Análise de Viabilidade

In [88]:
base_models = {'KNN': knn_base,
               'DT': dt_base,
               'LR': lr_base,
               'SVC': svc_base,
               'MLP': mlp_base}

models_results = {}
for model_name, base_model in tqdm(base_models.items()):
    mr = compute_feasibility(base_model, X, y, n_runs=3, n_splits=N_ITER, n_estimators=100)
    models_results[model_name] = mr

display(pd.DataFrame(models_results))

  0%|          | 0/5 [00:00<?, ?it/s]

### Treinamento Ensemble Não Balanceado

Definida a variável independente $\mathbf{X}$ e a variável alvo $\mathbf{y}$, podemos começar os treinamentos.

#### K-Nearest Neighbors (KNN)

In [22]:
knn_imb_bagging = BaggingClassifier(knn_base)

knn_imb_res = validate_ensemble(knn_imb_bagging, X, y, BAGGING_PARAM_GRID, targets_map=targets_map, n_iter=N_ITER, n_trainval_splits=TRAINVAL_SPLITS)
print_res(knn_imb_res)

  0%|          | 0/20 [00:00<?, ?it/s]

===> ACURÁCIA MÉDIA <===
(88.95 ± 3.48)%

===> MATRIZ DE CONFUSÃO GERAL <===
[[105   5   0   0   1  47]
 [  3 177   4   0   0   6]
 [  1   2 381   1   3   3]
 [  0   1   0  68   0   3]
 [  5   1   0   0  49   0]
 [ 26  15   4   4   5 348]]

===> RECALL, PRECISION E F1-SCORE MÉDIO <===
Target       | Recall       (%) | Precision    (%) | F1-Score     (%)
----------------------------------------------------------------------
Pastry       | 66.43 ± 12.41    | 76.83 ± 13.26    | 70.26 ± 9.28    
Z_Scratch    | 93.11 ± 6.33     | 88.87 ± 9.33     | 90.68 ± 6.29    
K_Scatch     | 97.47 ± 3.06     | 98.02 ± 2.49     | 97.7 ± 1.83     
Stains       | 94.58 ± 13.86    | 94.75 ± 11.41    | 93.54 ± 10.15   
Dirtiness    | 90.0 ± 19.04     | 89.83 ± 19.86    | 86.79 ± 16.37   
Bumps        | 86.56 ± 7.06     | 85.85 ± 6.73     | 86.0 ± 5.36     

===> MÉDIA DO RECALL MÉDIO <===
(88.03 ± 11.23)%

===> MÉDIA DO PRECISION MÉDIO <===
(89.02 ± 7.39)%

===> MÉDIA DO F1-SCORE MÉDIO <===
(87.49 ± 9.50)%


In [23]:
plot_accuracies(knn_imb_res, export=False, filename='knn_imb_acc', path='../figs/metrics_ensemble')

In [24]:
plot_label_metrics(knn_imb_res, 'f1-scores', export=False, filename='knn_imb_f1', path='../figs/metrics_ensemble')

In [25]:
plot_confusion_matrix(knn_imb_res, export=False, filename='knn_imb_cm', path='../figs/metrics_ensemble')

#### Árvore de Decisão

In [26]:
dt_imb_bagging = BaggingClassifier(dt_base)

dt_imb_res = validate_ensemble(dt_imb_bagging, X, y, BAGGING_PARAM_GRID, targets_map=targets_map, n_iter=N_ITER, n_trainval_splits=TRAINVAL_SPLITS)
print_res(dt_imb_res)

  0%|          | 0/20 [00:00<?, ?it/s]

===> ACURÁCIA MÉDIA <===
(91.25 ± 4.07)%

===> MATRIZ DE CONFUSÃO GERAL <===
[[125   1   0   0   2  30]
 [  1 179   2   0   0   8]
 [  3   2 379   0   0   7]
 [  0   0   0  67   0   5]
 [  4   0   0   0  47   4]
 [ 28   3   5   3   3 360]]

===> RECALL, PRECISION E F1-SCORE MÉDIO <===
Target       | Recall       (%) | Precision    (%) | F1-Score     (%)
----------------------------------------------------------------------
Pastry       | 79.02 ± 10.62    | 79.76 ± 15.45    | 78.69 ± 10.71   
Z_Scratch    | 94.0 ± 10.44     | 97.28 ± 5.76     | 95.14 ± 6.71    
K_Scatch     | 96.95 ± 4.2      | 98.28 ± 2.82     | 97.55 ± 2.59    
Stains       | 93.33 ± 14.46    | 96.5 ± 8.6       | 94.1 ± 10.19    
Dirtiness    | 85.0 ± 22.23     | 93.0 ± 15.17     | 86.65 ± 16.52   
Bumps        | 89.57 ± 7.92     | 87.1 ± 5.98      | 88.18 ± 6.12    

===> MÉDIA DO RECALL MÉDIO <===
(89.64 ± 6.64)%

===> MÉDIA DO PRECISION MÉDIO <===
(91.99 ± 7.24)%

===> MÉDIA DO F1-SCORE MÉDIO <===
(90.05 ± 6.97)%



In [27]:
plot_accuracies(dt_imb_res, export=False, filename='tree_imb_acc', path='../figs/metrics_ensemble')

In [28]:
plot_label_metrics(dt_imb_res, 'f1-scores', export=False, filename='tree_imb_f1', path='../figs/metrics_ensemble')

In [29]:
plot_confusion_matrix(dt_imb_res, export=False, filename='tree_bal_cm', path='../figs/metrics_ensemble')

#### Regressão Logística

Como alguns hiper-parâmetros não funcionam conjuntamente com outros (e.g. `solver=lbfgs` e `penalty=l1`), precisamos nos atentar a estes casos para evitar o treinamento destes modelos e suprimir possíveis erros. Para filtrar combinações inválidas:

In [30]:
#lr_imb_bagging = BaggingClassifier(lr_base)

#lr_imb_res = validate_ensemble(lr_imb_bagging, X, y, BAGGING_PARAM_GRID, targets_map=targets_map, n_iter=N_ITER, n_trainval_splits=TRAINVAL_SPLITS)
#print_res(lr_imb_res)

In [31]:
#plot_accuracies(lr_imb_res, export=False, filename='reglog_imb_acc', path='../figs/metrics_ensemble')

In [32]:
#plot_label_metrics(lr_imb_res, 'f1-scores', export=False, filename='reglog_imb_f1', path='../figs/metrics_ensemble')

In [33]:
#plot_confusion_matrix(lr_imb_res, export=False, filename='reglog_imb_cm', path='../figs/metrics_ensemble')

#### Support Vector Machine (SVM)

Assim como feito para a Regressão Logística, iremos filtrar os hiper-parâmetros do SVM que não podem/não fazem sentido ser combinados. Para este caso, apenas o grau quando o kernel é polinomial.

In [34]:
svc_imb_bagging = BaggingClassifier(svc_base)

svc_imb_res = validate_ensemble(svc_imb_bagging, X, y, BAGGING_PARAM_GRID, targets_map=targets_map, n_iter=N_ITER, n_trainval_splits=TRAINVAL_SPLITS)
print_res(svc_imb_res)

  0%|          | 0/20 [00:00<?, ?it/s]

===> ACURÁCIA MÉDIA <===
(91.09 ± 3.47)%

===> MATRIZ DE CONFUSÃO GERAL <===
[[124   2   2   0   2  28]
 [  0 177   6   0   0   7]
 [  3   2 382   0   0   4]
 [  0   0   0  68   0   4]
 [  5   0   0   0  48   2]
 [ 22  12   4   3   5 356]]

===> RECALL, PRECISION E F1-SCORE MÉDIO <===
Target       | Recall       (%) | Precision    (%) | F1-Score     (%)
----------------------------------------------------------------------
Pastry       | 78.57 ± 12.97    | 81.81 ± 12.62    | 79.53 ± 10.4    
Z_Scratch    | 93.0 ± 7.34      | 92.55 ± 9.28     | 92.45 ± 6.42    
K_Scatch     | 97.71 ± 3.06     | 97.05 ± 2.88     | 97.34 ± 2.24    
Stains       | 94.58 ± 13.86    | 96.5 ± 8.6       | 94.63 ± 9.38    
Dirtiness    | 87.5 ± 20.86     | 87.5 ± 18.44     | 86.07 ± 16.9    
Bumps        | 88.56 ± 7.58     | 89.0 ± 6.33      | 88.61 ± 5.91    

===> MÉDIA DO RECALL MÉDIO <===
(89.99 ± 6.76)%

===> MÉDIA DO PRECISION MÉDIO <===
(90.74 ± 5.82)%

===> MÉDIA DO F1-SCORE MÉDIO <===
(89.77 ± 6.45)%



In [35]:
plot_accuracies(svc_imb_res, export=False, filename='svc_imb_acc', path='../figs/metrics_ensemble')

In [36]:
plot_label_metrics(svc_imb_res, 'f1-scores', export=False, filename='svc_imb_f1', path='../figs/metrics_ensemble')

In [37]:
plot_confusion_matrix(svc_imb_res, export=False, filename='svc_imb_cm', path='../figs/metrics_ensemble')

#### Multi Layer Perceptron (MLP)

In [38]:
mlp_imb_bagging = BaggingClassifier(mlp_base)

mlp_imb_res = validate_ensemble(mlp_imb_bagging, X, y, BAGGING_PARAM_GRID, targets_map=targets_map, n_iter=N_ITER, n_trainval_splits=TRAINVAL_SPLITS)
print_res(mlp_imb_res)

  0%|          | 0/20 [00:00<?, ?it/s]

===> ACURÁCIA MÉDIA <===
(91.33 ± 3.41)%

===> MATRIZ DE CONFUSÃO GERAL <===
[[126   2   0   1   1  28]
 [  1 176   5   0   0   8]
 [  2   2 380   3   0   4]
 [  0   0   0  69   0   3]
 [  4   0   0   0  47   4]
 [ 22   9   6   2   3 360]]

===> RECALL, PRECISION E F1-SCORE MÉDIO <===
Target       | Recall       (%) | Precision    (%) | F1-Score     (%)
----------------------------------------------------------------------
Pastry       | 79.82 ± 14.93    | 82.06 ± 11.45    | 80.24 ± 11.02   
Z_Scratch    | 92.5 ± 8.81      | 93.78 ± 6.86     | 92.72 ± 5.29    
K_Scatch     | 97.18 ± 3.08     | 97.26 ± 2.55     | 97.18 ± 2.02    
Stains       | 95.83 ± 10.3     | 93.25 ± 12.49    | 93.8 ± 9.23     
Dirtiness    | 85.83 ± 21.13    | 92.92 ± 15.12    | 87.29 ± 15.85   
Bumps        | 89.56 ± 6.61     | 88.67 ± 6.32     | 88.99 ± 5.64    

===> MÉDIA DO RECALL MÉDIO <===
(90.12 ± 6.53)%

===> MÉDIA DO PRECISION MÉDIO <===
(91.32 ± 5.30)%

===> MÉDIA DO F1-SCORE MÉDIO <===
(90.04 ± 5.95)%



In [39]:
plot_accuracies(mlp_imb_res, export=False, filename='mlp_imb_acc', path='../figs/metrics_ensemble')

In [40]:
plot_label_metrics(mlp_imb_res, 'f1-scores', export=False, filename='mlp_imb_f1', path='../figs/metrics_ensemble')

In [41]:
plot_confusion_matrix(mlp_imb_res, export=False, filename='mlp_imb_cm', path='../figs/metrics_ensemble')

### Treinamento Ensemble Balanceado

#### K-Nearest Neighbors (KNN)

In [42]:
knn_bal_bagging = BalancedBaggingClassifier(knn_base, sampling_strategy='not minority')

knn_bal_res = validate_ensemble(knn_bal_bagging, X, y, BAGGING_PARAM_GRID, targets_map=targets_map, n_iter=N_ITER, n_trainval_splits=TRAINVAL_SPLITS)
print_res(knn_bal_res)

  0%|          | 0/20 [00:00<?, ?it/s]

===> ACURÁCIA MÉDIA <===
(82.88 ± 5.14)%

===> MATRIZ DE CONFUSÃO GERAL <===
[[119   7   1   2  15  14]
 [  3 173   3   2   7   2]
 [  1   7 370   3   3   7]
 [  0   1   0  70   0   1]
 [  1   1   0   0  53   0]
 [ 39  54   4  15  24 266]]

===> RECALL, PRECISION E F1-SCORE MÉDIO <===
Target       | Recall       (%) | Precision    (%) | F1-Score     (%)
----------------------------------------------------------------------
Pastry       | 75.45 ± 16.33    | 74.12 ± 14.4     | 73.74 ± 12.83   
Z_Scratch    | 90.94 ± 6.25     | 72.53 ± 10.25    | 80.24 ± 6.7     
K_Scatch     | 94.66 ± 4.78     | 97.98 ± 2.54     | 96.2 ± 2.36     
Stains       | 97.08 ± 9.08     | 80.19 ± 18.04    | 86.22 ± 10.67   
Dirtiness    | 96.67 ± 14.91    | 54.9 ± 17.63     | 68.72 ± 16.3    
Bumps        | 66.11 ± 12.18    | 91.68 ± 8.44     | 76.37 ± 10.11   

===> MÉDIA DO RECALL MÉDIO <===
(86.82 ± 12.95)%

===> MÉDIA DO PRECISION MÉDIO <===
(78.57 ± 15.28)%

===> MÉDIA DO F1-SCORE MÉDIO <===
(80.25 ± 9.80)%

In [43]:
plot_accuracies(knn_bal_res, export=False, filename='knn_bal_acc', path='../figs/metrics_ensemble')

In [44]:
plot_label_metrics(knn_bal_res, 'f1-scores', export=False, filename='knn_bal_f1', path='../figs/metrics_ensemble')

In [45]:
plot_confusion_matrix(knn_bal_res, export=False, filename='knn_bal_cm', path='../figs/metrics_ensemble')

#### Árvore de Decisão

In [46]:
dt_bal_bagging = BalancedBaggingClassifier(dt_base, sampling_strategy='not minority')

dt_bal_res = validate_ensemble(dt_bal_bagging, X, y, BAGGING_PARAM_GRID, targets_map=targets_map, n_iter=N_ITER, n_trainval_splits=TRAINVAL_SPLITS)
print_res(dt_bal_res)

  0%|          | 0/20 [00:00<?, ?it/s]

===> ACURÁCIA MÉDIA <===
(88.40 ± 3.83)%

===> MATRIZ DE CONFUSÃO GERAL <===
[[134   2   0   1   5  16]
 [  3 174   4   0   2   7]
 [  5   1 373   2   0  10]
 [  0   0   0  67   0   5]
 [  4   1   0   0  47   3]
 [ 44  13   8   2   9 326]]

===> RECALL, PRECISION E F1-SCORE MÉDIO <===
Target       | Recall       (%) | Precision    (%) | F1-Score     (%)
----------------------------------------------------------------------
Pastry       | 84.73 ± 12.06    | 72.32 ± 14.06    | 77.39 ± 11.21   
Z_Scratch    | 91.44 ± 8.31     | 92.22 ± 9.31     | 91.36 ± 6.12    
K_Scatch     | 95.42 ± 3.99     | 97.02 ± 2.94     | 96.13 ± 1.97    
Stains       | 93.33 ± 14.46    | 94.25 ± 10.29    | 92.83 ± 10.17   
Dirtiness    | 85.83 ± 21.13    | 80.0 ± 21.01     | 79.87 ± 17.41   
Bumps        | 81.07 ± 6.97     | 89.49 ± 7.85     | 84.78 ± 5.38    

===> MÉDIA DO RECALL MÉDIO <===
(88.64 ± 5.59)%

===> MÉDIA DO PRECISION MÉDIO <===
(87.55 ± 9.48)%

===> MÉDIA DO F1-SCORE MÉDIO <===
(87.06 ± 7.54)%



In [47]:
plot_accuracies(dt_bal_res, export=False, filename='tree_bal_acc', path='../figs/metrics_ensemble')

In [48]:
plot_label_metrics(dt_bal_res, 'f1-scores', export=False, filename='tree_bal_f1', path='../figs/metrics_ensemble')

In [49]:
plot_confusion_matrix(dt_bal_res, export=False, filename='tree_bal_cm', path='../figs/metrics_ensemble')

#### Regressão Logística

Como alguns hiper-parâmetros não funcionam conjuntamente com outros (e.g. `solver=lbfgs` e `penalty=l1`), precisamos nos atentar a estes casos para evitar o treinamento destes modelos e suprimir possíveis erros. Para filtrar combinações inválidas:

In [50]:
#lr_bal_bagging = BalancedBaggingClassifier(lr_base, sampling_strategy='not minority')

#lr_bal_res = validate_ensemble(lr_bal_bagging, X, y, BAGGING_PARAM_GRID, targets_map=targets_map, n_iter=N_ITER, n_trainval_splits=TRAINVAL_SPLITS)
#print_res(lr_bal_res)

In [51]:
#plot_accuracies(lr_bal_res, export=False, filename='reglog_bal_acc', path='../figs/metrics_ensemble')

In [52]:
#plot_label_metrics(lr_bal_res, 'f1-scores', export=False, filename='reglog_bal_f1', path='../figs/metrics_ensemble')

In [53]:
#plot_confusion_matrix(lr_bal_res, export=False, filename='reglog_bal_cm', path='../figs/metrics_ensemble')

#### Support Vector Machine (SVM)

Assim como feito para a Regressão Logística, iremos filtrar os hiper-parâmetros do SVM que não podem/não fazem sentido ser combinados. Para este caso, apenas o grau quando o kernel é polinomial.

In [62]:
svc_bal_bagging = BalancedBaggingClassifier(svc_base, sampling_strategy='not minority')
#from imblearn.over_sampling import SMOTE
#svc_bal_bagging = BalancedBaggingClassifier(svc_base, sampling_strategy='not majority', sampler=SMOTE())
#svc_bal_bagging = BalancedBaggingClassifier(svc_base, sampling_strategy='not majority', sampler=SMOTE())

svc_bal_res = validate_ensemble(svc_bal_bagging, X, y, BAGGING_PARAM_GRID, targets_map=targets_map, n_iter=N_ITER, n_trainval_splits=TRAINVAL_SPLITS)
print_res(svc_bal_res)

  0%|          | 0/20 [00:00<?, ?it/s]

===> ACURÁCIA MÉDIA <===
(87.54 ± 4.21)%

===> MATRIZ DE CONFUSÃO GERAL <===
[[137   3   0   0   3  15]
 [  5 179   4   0   0   2]
 [  2   9 372   2   0   6]
 [  0   0   0  69   0   3]
 [  5   0   0   0  50   0]
 [ 53  20   6   9  11 303]]

===> RECALL, PRECISION E F1-SCORE MÉDIO <===
Target       | Recall       (%) | Precision    (%) | F1-Score     (%)
----------------------------------------------------------------------
Pastry       | 86.79 ± 14.33    | 69.5 ± 12.31     | 76.15 ± 9.2     
Z_Scratch    | 94.22 ± 6.47     | 86.31 ± 11.63    | 89.62 ± 7.08    
K_Scatch     | 95.14 ± 4.85     | 97.53 ± 2.94     | 96.22 ± 2.48    
Stains       | 95.83 ± 10.3     | 89.17 ± 14.37    | 91.23 ± 8.8     
Dirtiness    | 90.83 ± 19.85    | 79.58 ± 19.4     | 83.17 ± 16.68   
Bumps        | 75.37 ± 11.4     | 92.62 ± 7.04     | 82.51 ± 7.76    

===> MÉDIA DO RECALL MÉDIO <===
(89.70 ± 7.78)%

===> MÉDIA DO PRECISION MÉDIO <===
(85.78 ± 10.00)%

===> MÉDIA DO F1-SCORE MÉDIO <===
(86.48 ± 7.22)%


In [63]:
plot_accuracies(svc_bal_res, export=False, filename='svc_bal_acc', path='../figs/metrics_ensemble')

In [64]:
plot_label_metrics(svc_bal_res, 'f1-scores', export=False, filename='svc_bal_f1', path='../figs/metrics_ensemble')

In [65]:
plot_confusion_matrix(svc_bal_res, export=False, filename='svc_bal_cm', path='../figs/metrics_ensemble')

#### Multi Layer Perceptron (MLP)

In [58]:
mlp_bal_bagging = BalancedBaggingClassifier(mlp_base, sampling_strategy='not minority')

mlp_bal_res = validate_ensemble(mlp_bal_bagging, X, y, BAGGING_PARAM_GRID, targets_map=targets_map, n_iter=N_ITER, n_trainval_splits=TRAINVAL_SPLITS)
print_res(mlp_bal_res)

  0%|          | 0/20 [00:00<?, ?it/s]

===> ACURÁCIA MÉDIA <===
(83.35 ± 4.90)%

===> MATRIZ DE CONFUSÃO GERAL <===
[[120   9   2   2  12  13]
 [  2 165   7   2   8   6]
 [  1   2 375   5   0   8]
 [  0   1   0  70   0   1]
 [  5   1   0   0  47   2]
 [ 50  26   7  18  21 280]]

===> RECALL, PRECISION E F1-SCORE MÉDIO <===
Target       | Recall       (%) | Precision    (%) | F1-Score     (%)
----------------------------------------------------------------------
Pastry       | 75.98 ± 16.45    | 68.46 ± 11.88    | 71.24 ± 11.81   
Z_Scratch    | 86.67 ± 9.92     | 82.08 ± 11.71    | 83.81 ± 8.66    
K_Scatch     | 95.89 ± 4.62     | 96.05 ± 2.99     | 95.88 ± 2.45    
Stains       | 97.08 ± 9.08     | 76.55 ± 17.38    | 83.93 ± 10.44   
Dirtiness    | 85.83 ± 21.13    | 57.56 ± 23.02    | 66.79 ± 19.24   
Bumps        | 69.64 ± 10.51    | 90.41 ± 7.68     | 78.31 ± 8.95    

===> MÉDIA DO RECALL MÉDIO <===
(85.18 ± 10.82)%

===> MÉDIA DO PRECISION MÉDIO <===
(78.52 ± 14.18)%

===> MÉDIA DO F1-SCORE MÉDIO <===
(79.99 ± 10.36)

In [59]:
plot_accuracies(mlp_bal_res, export=False, filename='mlp_bal_acc', path='../figs/metrics_ensemble')

In [60]:
plot_label_metrics(mlp_bal_res, 'f1-scores', export=False, filename='mlp_bal_f1', path='../figs/metrics_ensemble')

In [61]:
plot_confusion_matrix(mlp_bal_res, export=False, filename='mlp_bal_cm', path='../figs/metrics_ensemble')

---

## Playground

In [ ]:
from imblearn.under_sampling import CondensedNearestNeighbour, EditedNearestNeighbours
from imblearn.pipeline import Pipeline as ImbPipeline

In [ ]:
svc_bal_bagging = BalancedBaggingClassifier(svc_base, sampling_strategy='not minority', sampler=EditedNearestNeighbours())

svc_bal_res = validate_ensemble(svc_bal_bagging, X, y, BAGGING_PARAM_GRID, targets_map=targets_map, n_iter=N_ITER, n_trainval_splits=TRAINVAL_SPLITS)
print_res(svc_bal_res)

In [ ]:
svc_bal_bagging = BalancedBaggingClassifier(svc_base, sampling_strategy='not minority', sampler=CondensedNearestNeighbour())

svc_bal_res = validate_ensemble(svc_bal_bagging, X, y, BAGGING_PARAM_GRID, targets_map=targets_map, n_iter=N_ITER, n_trainval_splits=TRAINVAL_SPLITS)
print_res(svc_bal_res)

In [ ]:
enn_cnn_sampler = ImbPipeline([('enn', EditedNearestNeighbours()), ('cnn', CondensedNearestNeighbour())])

svc_bal_bagging = BalancedBaggingClassifier(svc_base, sampling_strategy='not minority', sampler=enn_cnn_sampler)

svc_bal_res = validate_ensemble(svc_bal_bagging, X, y, BAGGING_PARAM_GRID, targets_map=targets_map, n_iter=N_ITER, n_trainval_splits=TRAINVAL_SPLITS)
print_res(svc_bal_res)